In [1]:
import os
import pandas as pd
pd.set_option("display.max_columns", 50)
import matplotlib.pyplot as plt
import warnings
import plotly.express as px
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=UserWarning)
from pyhive import hive

In [2]:
username = os.environ['USERNAME']
hive_host = os.environ['HIVE_SERVER2'].split(':')[0]
hive_port = os.environ['HIVE_SERVER2'].split(':')[1]

In [3]:
print(hive_host)

iccluster044.iccluster.epfl.ch


In [4]:
conn = hive.connect(host=hive_host, port=hive_port)

In [5]:
cur = conn.cursor()

print(f"your username is {username}")
print(f"you are connected to {hive_host}:{hive_port}")

your username is tcyu
you are connected to iccluster044.iccluster.epfl.ch:10000


In [ ]:
query = f"""
SELECT * FROM {username}.sbb_trips_orc LIMIT 5
"""
pd.read_sql(query,conn)

,sbb_trips_orc.route_id,sbb_trips_orc.service_id,sbb_trips_orc.trip_id,sbb_trips_orc.trip_headsign,sbb_trips_orc.trip_short_name,sbb_trips_orc.direction_id
0,1-1-C-j18-1,TA+b0001,9.TA.1-1-C-j18-1.4.R,"Zofingen, Altachen",108,1
1,1-1-C-j18-1,TA+b0001,11.TA.1-1-C-j18-1.4.R,"Zofingen, Altachen",112,1
2,1-1-C-j18-1,TA+b0001,13.TA.1-1-C-j18-1.4.R,"Zofingen, Altachen",116,1
3,1-1-C-j18-1,TA+b0001,15.TA.1-1-C-j18-1.4.R,"Zofingen, Altachen",120,1
4,1-1-C-j18-1,TA+b0001,17.TA.1-1-C-j18-1.4.R,"Zofingen, Altachen",124,1


In [12]:
# filter for stops in a 15km radius of ZH train station (lat, lon) = (47.378177, 8.540192)
query_filter_stops_near_zh = f"""
    CREATE TABLE {username}.filtered_stops
    STORED AS ORC
    AS
    (SELECT *
    FROM {username}.sbb_stops_orc
    WHERE (6371 * acos(cos(radians(STOP_LAT)) * cos(radians(47.378177))
                    * cos(radians(8.540192) - radians(STOP_LON))
                    + sin(radians(STOP_LAT)) * sin(radians(47.378177)))) <= 15)
"""
cur.execute(query)

In [13]:
query_filter_stops_near_zh = f"""
    SELECT *
    FROM {username}.filtered_stops
    LIMIT 5
"""
stops_near_zh_central = pd.read_sql(query_filter_stops_near_zh, conn)
stops_near_zh_central

,filtered_stops.stop_id,filtered_stops.stop_name,filtered_stops.stop_lat,filtered_stops.stop_lon,filtered_stops.location_type,filtered_stops.parent_station
0,176,Zimmerberg-Basistunnel,47.351677,8.521957,,
1,8500926,"Oetwil a.d.L., Schweizäcker",47.423626,8.403183,,
2,8502075,"Zürich Flughafen, Carterminal",47.451023,8.563729,,
3,8502186,Dietikon Stoffelbach,47.393326,8.398960,,Parent8502186
4,8502186:0,Dietikon Stoffelbach,47.393400,8.398943,,Parent8502186


In [ ]:
query = f"""
CREATE TABLE {username}.filtered_stop_times
AS
SELECT
    a.arrival_time,
    a.departure_time,
    a.stop_id,
    a.stop_sequence,
    a.pickup_type,
    a.drop_off_type
FROM {username}.sbb_stop_times_orc a LEFT SEMI JOIN {username}.filtered_stops b
ON  a.stop_id = b.stop_id
"""
cur.execute(query)

### Join trips with stop times

In [14]:
query = f"""
SELECT
    *
FROM {username}.trip_with_stop_times
LIMIT 5
"""
# cur.execute(query)
pd.read_sql(query,conn)

,trip_with_stop_times.route_id,trip_with_stop_times.service_id,trip_with_stop_times.trip_id,trip_with_stop_times.trip_headsign,trip_with_stop_times.trip_short_name,trip_with_stop_times.direction_id,trip_with_stop_times.arrival_time,trip_with_stop_times.departure_time,trip_with_stop_times.stop_id,trip_with_stop_times.stop_sequence,trip_with_stop_times.pickup_type,trip_with_stop_times.drop_off_type
0,96-181-3-j21-1,TA+lg320,88.TA.96-181-3-j21-1.3.H,"Zürich Enge, Bahnhof/Bederstr.",26956,0,18:30:00,18:30:00,8591059,8,0,0
1,96-181-3-j21-1,TA+w1000,88.TA.96-181-3-j21-1.3.H,"Zürich Enge, Bahnhof/Bederstr.",10914,0,18:30:00,18:30:00,8591059,8,0,0
2,96-181-3-j21-1,TA+12000,88.TA.96-181-3-j21-1.3.H,"Zürich Enge, Bahnhof/Bederstr.",9364,0,18:30:00,18:30:00,8591059,8,0,0
3,96-181-3-j21-1,TA+w1000,88.TA.96-181-3-j21-1.3.H,"Zürich Enge, Bahnhof/Bederstr.",10914,0,18:30:00,18:30:00,8591059,8,0,0
4,96-181-3-j21-1,TA+r4o00,88.TA.96-181-3-j21-1.3.H,"Zürich Enge, Bahnhof/Bederstr.",35870,0,18:30:00,18:30:00,8591059,8,0,0


### Make connections table including all possible connections

In [ ]:
query = f"""
SELECT s.trip_id FROM (SELECT * FROM {username}.filtered_stop_times LIMIT 100) s
"""
# cur.execute(query)
stops = pd.read_sql(query,conn)

In [27]:
d = {'col1': [1, 2], 'col2': [3, 4]}
df = pd.DataFrame(data=d)
for r in df:
    trip_id = r['trip_id']
    query = f"""
            SELECT
                *
            FROM {username}.filtered_stop_times
            WHERE trip_id = {trip_id}
            ORDER BY stop_sequence
            """
    trips = pd.read_sql(query, conn)
    for i, o in trips.iterrows():
        if t['pickup_type'] != 1:
            for d in trips[i + 1:]:
                if d['drop_off_type'] != 1:
                    query = f"""
                            INSERT connections VALUES
                            ({o['stop_id']}, {d['stop_id']}, {o['departure_time']}, {d['arrival_time']}, {trip_id})
                            """
                if d['pickup_type'] != 1:
                    break

1
2


In [24]:
import numpy as np

In [41]:
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
                   columns=['a', 'b', 'c'])
print(df)

   a  b  c
0  1  2  3
1  4  5  6
2  7  8  9


In [ ]:
stops.head

### Make footpath

In [34]:
from scipy.sparse import csr_matrix, eye
from datetime import datetime, timedelta

In [6]:
query = f"""
SELECT 
    stop_id,
    stop_lat,
    stop_lon
FROM {username}.filtered_stops
"""
# cur.execute(query)
stops = pd.read_sql(query,conn)

In [17]:
def time(minutes):
    return datetime(year=2023, month=5, day=16, hour=12, minute=00) + timedelta(minutes=minutes)

In [139]:
def is_reachable(T, S, c):
    return T[c['trip_id']] is not None or S[c['departure_stop']] < c['departure_time']

In [168]:
def connection_scan(source, target, start_time, num_stops, num_trips, footpaths, connections):
    S = [datetime.max] * num_stops
    T = [None] * num_trips
    J = [None] * num_stops
    for i, f in enumerate(footpaths[source]):
        if f != 0:
            S[i] = start_time + timedelta(minutes=int(f))
    
    for _, c in connections.iterrows():
        if S[target] <= c['departure_time']:
            break
        
        if is_reachable(T, S, c):
            if T[c['trip_id']] is None:
                T[c['trip_id']] = c
            for i, f in enumerate(footpaths[c['arrival_stop']]):
                if f != 0:
                    if c['arrival_time'] + timedelta(minutes=int(f)) < S[i]:
                        S[i] = c['arrival_time'] + timedelta(minutes=int(f))
                        J[i] = (T[c['trip_id']], c, f)
        print(S)
        print()
    return J

def get_journey(source, target, J, footpaths):
    journey = []
    t = target
    while J[t] is not None:
        journey = [J[t]] + journey
        t = J[t][0]['departure_stop']
    if footpaths[source][target] != 0:
        journey = [footpaths[source][target]] + journey
    return journey

In [177]:
e_c = [
    ['bus', 1, 3, time(15), time(18), 1, 0],
    ['train', 1, 2, time(13), time(15), 0, 1],
    ['bus', 0, 1, time(10), time(15), 1, 0],
    ['train', 4, 1, time(9), time(13), 0, 1],
    ['bus', 6, 0, time(8), time(10), 1, 0],
    ['train', 5, 4, time(7), time(12), 0, 1],
]

# what if arrival and departure times are the same

df_connections = pd.DataFrame(
    e_c, columns=['route_desc', 'departure_stop', 'arrival_stop', 'departure_time',
                  'arrival_time', 'trip_id', 'distribution_id']
).sort_values(by='departure_time')

# footpaths = np.array([
#     [1, 0, 0, 0, 0, 0, 0],
#     [0, 1, 0, 0, 0, 0, 0],
#     [0, 0, 1, 2, 0, 0, 0],
#     [0, 0, 2, 1, 0, 0, 0],
#     [0, 0, 0, 0, 1, 0, 0],
#     [0, 0, 0, 0, 0, 1, 2],
#     [0, 0, 0, 0, 0, 2, 1],
# ])
footpaths = np.array([
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 1],
])

source = 5
destination = 3

In [175]:
J = connection_scan(source, destination, time(5), 7, 2, footpaths, df_connections)

[datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), Timestamp('2023-05-16 12:13:00'), datetime.datetime(2023, 5, 16, 12, 6), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999)]

[datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), Timestamp('2023-05-16 12:13:00'), datetime.datetime(2023, 5, 16, 12, 6), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999)]

[datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), Timestamp('2023-05-16 12:14:00'), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999), Timestamp('2023-05-16 12:13:00'), datetime.datetime(2023, 5, 16, 12, 6), datetime.datetime(9999, 12, 31, 23, 59, 59, 999999)]


In [172]:
print(J[3][2])

2


In [176]:
get_journey(source, destination, J, footpaths)

[(route_desc                       train
  departure_stop                       5
  arrival_stop                         4
  departure_time     2023-05-16 12:07:00
  arrival_time       2023-05-16 12:12:00
  trip_id                              0
  distribution_id                      1
  Name: 5, dtype: object,
  route_desc                       train
  departure_stop                       4
  arrival_stop                         1
  departure_time     2023-05-16 12:09:00
  arrival_time       2023-05-16 12:13:00
  trip_id                              0
  distribution_id                      1
  Name: 3, dtype: object,
  1),
 (route_desc                         bus
  departure_stop                       1
  arrival_stop                         3
  departure_time     2023-05-16 12:15:00
  arrival_time       2023-05-16 12:18:00
  trip_id                              1
  distribution_id                      0
  Name: 0, dtype: object,
  route_desc                         bus
  departure_st